## End of Day Position Calculation Process

### Importing relevant libraries

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir()

['.ipynb_checkpoints',
 '1537277231233_Input_Transactions.txt',
 'disclaim.txt',
 'Expected_EndOfDay_Positions.txt',
 'Input_StartOfDay_Positions.txt',
 'Position_Calculation.ipynb',
 'Sample_Illustration.txt']

### Importing Start of Day Position File

In [3]:
positions=pd.read_csv('Input_StartOfDay_Positions.txt')
positions.head()

,Instrument,Account,AccountType,Quantity
0,IBM,101,E,100000
1,IBM,201,I,-100000
2,MSFT,101,E,5000000
3,MSFT,201,I,-5000000
4,APPL,101,E,10000


### Importing Transaction File for the Day

In [4]:
# Importing Transaction File
transactions=pd.read_json('1537277231233_Input_Transactions.txt')
transactions.head()

,Instrument,TransactionId,TransactionQuantity,TransactionType
0,IBM,1,1000,B
1,APPL,2,200,S
2,AMZN,3,5000,S
3,MSFT,4,50,B
4,APPL,5,100,B


### Processing Transaction File

##### Since Buy and Sell are opposite type of transactions, assigning -ve sign to sells and assigning to New Column: Net_Transaction

In [5]:
transactions['Net_Transaction']=transactions.TransactionType.replace({'B':1,'S':-1}) * transactions.TransactionQuantity
transactions.head()

,Instrument,TransactionId,TransactionQuantity,TransactionType,Net_Transaction
0,IBM,1,1000,B,1000
1,APPL,2,200,S,-200
2,AMZN,3,5000,S,-5000
3,MSFT,4,50,B,50
4,APPL,5,100,B,100


##### Grouping Transactions by Instrument and finding Net Change(Here +ve means Net Transaction is Buy, reverse is Sell)

In [6]:
transactions=(transactions
              .groupby(['Instrument'])
              .agg({'Net_Transaction':np.sum})
              .reset_index())
transactions.head()

,Instrument,Net_Transaction
0,AMZN,-14850
1,APPL,-11100
2,IBM,1000
3,MSFT,-250


### * There may be Transaction entries for instruments that are not present in Position File
#### So, including two entries in the position file for any such instruments(E & I with Quantity as Zero)

In [7]:
new_instruments=[ x for x in transactions.Instrument if x not in positions.Instrument.unique()]
if(len(new_instruments)>0):
    for inst in new_instruments:
        positions=positions.append([{'Instrument':inst,'Account':101,'AccountType':'E','Quantity':0},
                                    {'Instrument':inst,'Account':201,'AccountType':'I','Quantity':0}],
                                   ignore_index=True)

positions.head()
         

,Instrument,Account,AccountType,Quantity
0,IBM,101,E,100000
1,IBM,201,I,-100000
2,MSFT,101,E,5000000
3,MSFT,201,I,-5000000
4,APPL,101,E,10000


### Merging Positions File and Transactions File

In [8]:
# Left Join as Some Instruments may not have Transactions in the Day, but may have Previous Balance in Positions
# We have already taken care for reverse scenraio in previous step

positions_eod=pd.merge(positions,transactions,how='left',on='Instrument')

# Imputing NaN with Zero
positions_eod.Net_Transaction.fillna(0,inplace=True)

# Converting the column Back to integer
positions_eod.Net_Transaction=positions_eod.Net_Transaction.astype(int)

positions_eod.head()

,Instrument,Account,AccountType,Quantity,Net_Transaction
0,IBM,101,E,100000,1000
1,IBM,201,I,-100000,1000
2,MSFT,101,E,5000000,-250
3,MSFT,201,I,-5000000,-250
4,APPL,101,E,10000,-11100


#### Computing the New Quantity
##### (for AccountType 'E' : Quantity+Net_Transaction,  for AccountType 'I' : Quantity-Net_Transaction)

In [9]:
# Creating New Column which takes care of (+) and (-) for Accounttype E & I (This Delta is Volume Change)
positions_eod['Delta']=np.where(positions_eod.AccountType == 'E',positions_eod.Net_Transaction,-positions_eod.Net_Transaction)
positions_eod['Quantity']=positions_eod.Quantity + positions_eod.Delta
del positions_eod['Net_Transaction']

## Final Output
positions_eod.head()


,Instrument,Account,AccountType,Quantity,Delta
0,IBM,101,E,101000,1000
1,IBM,201,I,-101000,-1000
2,MSFT,101,E,4999750,-250
3,MSFT,201,I,-4999750,250
4,APPL,101,E,-1100,-11100


### Instruments with Max and Min Volume Change for the Day (Considered Absolute Values for Min Max)

In [10]:
# I have considered absolute value of Delta to Calculate max and min Volumes

Volumes_max_min={'MaxVolume':positions_eod.Instrument[positions_eod.Delta.abs()==np.max(positions_eod.Delta.abs())].unique(),
                 "MinVolume":positions_eod.Instrument[positions_eod.Delta.abs()==np.min(positions_eod.Delta.abs())].unique()}

Volumes_max_min


{'MaxVolume': array(['AMZN'], dtype=object),
 'MinVolume': array(['NFLX'], dtype=object)}

### Reading Expected Output File & Matching with my Calculated Output

In [11]:

output=pd.read_csv('Expected_EndOfDay_Positions.txt')

positions_eod==output

,Instrument,Account,AccountType,Quantity,Delta
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,True,True,True,True,True
6,True,True,True,True,True
7,True,True,True,True,True
8,True,True,True,True,True
9,True,True,True,True,True
